# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 2 2022 8:30AM,243202,10,Enova-10394,Emerginnova,Released
1,Aug 2 2022 8:30AM,243202,10,Enova-10395,Emerginnova,Released
2,Aug 2 2022 8:30AM,243203,10,MSP212713,"Methapharm, Inc.",Released
3,Aug 2 2022 8:30AM,243205,10,0085919771,ISDIN Corporation,Released
4,Aug 2 2022 8:30AM,243205,10,0085919772,ISDIN Corporation,Released
5,Aug 2 2022 8:30AM,243205,10,0085919831,ISDIN Corporation,Released
6,Aug 2 2022 8:30AM,243205,10,0085919832,ISDIN Corporation,Released
7,Aug 2 2022 8:30AM,243205,10,0085919833,ISDIN Corporation,Released
8,Aug 2 2022 8:30AM,243205,10,0085919835,ISDIN Corporation,Released
9,Aug 1 2022 3:53PM,243209,19,GP08032022,"Granules Pharmaceuticals, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,243203,Released,1
26,243204,Released,19
27,243205,Released,6
28,243206,Released,1
29,243209,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243203,NaN,NaN,1.0
243204,NaN,NaN,19.0
243205,NaN,NaN,6.0
243206,NaN,NaN,1.0
243209,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243085,0.0,0.0,1.0
243110,0.0,0.0,3.0
243123,0.0,2.0,0.0
243129,0.0,0.0,1.0
243145,2.0,34.0,11.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243085,0,0,1
243110,0,0,3
243123,0,2,0
243129,0,0,1
243145,2,34,11


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243085,0,0,1
1,243110,0,0,3
2,243123,0,2,0
3,243129,0,0,1
4,243145,2,34,11


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243085,,,1
1,243110,,,3
2,243123,,2,
3,243129,,,1
4,243145,2,34,11


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 2 2022 8:30AM,243202,10,Emerginnova
2,Aug 2 2022 8:30AM,243203,10,"Methapharm, Inc."
3,Aug 2 2022 8:30AM,243205,10,ISDIN Corporation
9,Aug 1 2022 3:53PM,243209,19,"Granules Pharmaceuticals, Inc."
10,Aug 1 2022 3:45PM,243206,19,"AdvaGen Pharma, Ltd"
11,Aug 1 2022 3:40PM,243204,19,"AdvaGen Pharma, Ltd"
30,Aug 1 2022 3:13PM,243200,10,"Nextsource Biotechnology, LLC"
31,Aug 1 2022 3:07PM,243199,10,Yusen – 3D Matrix
32,Aug 1 2022 2:36PM,243193,19,"AdvaGen Pharma, Ltd"
40,Aug 1 2022 2:08PM,243185,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 2 2022 8:30AM,243202,10,Emerginnova,,,2
1,Aug 2 2022 8:30AM,243203,10,"Methapharm, Inc.",,,1
2,Aug 2 2022 8:30AM,243205,10,ISDIN Corporation,,,6
3,Aug 1 2022 3:53PM,243209,19,"Granules Pharmaceuticals, Inc.",,,1
4,Aug 1 2022 3:45PM,243206,19,"AdvaGen Pharma, Ltd",,,1
5,Aug 1 2022 3:40PM,243204,19,"AdvaGen Pharma, Ltd",,,19
6,Aug 1 2022 3:13PM,243200,10,"Nextsource Biotechnology, LLC",,,1
7,Aug 1 2022 3:07PM,243199,10,Yusen – 3D Matrix,,,1
8,Aug 1 2022 2:36PM,243193,19,"AdvaGen Pharma, Ltd",,7,1
9,Aug 1 2022 2:08PM,243185,10,"Methapharm, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 8:30AM,243202,10,Emerginnova,2,,
1,Aug 2 2022 8:30AM,243203,10,"Methapharm, Inc.",1,,
2,Aug 2 2022 8:30AM,243205,10,ISDIN Corporation,6,,
3,Aug 1 2022 3:53PM,243209,19,"Granules Pharmaceuticals, Inc.",1,,
4,Aug 1 2022 3:45PM,243206,19,"AdvaGen Pharma, Ltd",1,,
5,Aug 1 2022 3:40PM,243204,19,"AdvaGen Pharma, Ltd",19,,
6,Aug 1 2022 3:13PM,243200,10,"Nextsource Biotechnology, LLC",1,,
7,Aug 1 2022 3:07PM,243199,10,Yusen – 3D Matrix,1,,
8,Aug 1 2022 2:36PM,243193,19,"AdvaGen Pharma, Ltd",1,7,
9,Aug 1 2022 2:08PM,243185,10,"Methapharm, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 8:30AM,243202,10,Emerginnova,2,,
1,Aug 2 2022 8:30AM,243203,10,"Methapharm, Inc.",1,,
2,Aug 2 2022 8:30AM,243205,10,ISDIN Corporation,6,,
3,Aug 1 2022 3:53PM,243209,19,"Granules Pharmaceuticals, Inc.",1,,
4,Aug 1 2022 3:45PM,243206,19,"AdvaGen Pharma, Ltd",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 8:30AM,243202,10,Emerginnova,2.0,NaN,NaN
1,Aug 2 2022 8:30AM,243203,10,"Methapharm, Inc.",1.0,NaN,NaN
2,Aug 2 2022 8:30AM,243205,10,ISDIN Corporation,6.0,NaN,NaN
3,Aug 1 2022 3:53PM,243209,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
4,Aug 1 2022 3:45PM,243206,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 8:30AM,243202,10,Emerginnova,2.0,0.0,0.0
1,Aug 2 2022 8:30AM,243203,10,"Methapharm, Inc.",1.0,0.0,0.0
2,Aug 2 2022 8:30AM,243205,10,ISDIN Corporation,6.0,0.0,0.0
3,Aug 1 2022 3:53PM,243209,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
4,Aug 1 2022 3:45PM,243206,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2431802,22.0,2.0,1.0
15,243154,2.0,17.0,1.0
16,243123,0.0,2.0,0.0
19,1459093,24.0,8.0,0.0
20,729400,50.0,41.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2431802,22.0,2.0,1.0
1,15,243154,2.0,17.0,1.0
2,16,243123,0.0,2.0,0.0
3,19,1459093,24.0,8.0,0.0
4,20,729400,50.0,41.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,22.0,2.0,1.0
1,15,2.0,17.0,1.0
2,16,0.0,2.0,0.0
3,19,24.0,8.0,0.0
4,20,50.0,41.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,22.0
1,15,Released,2.0
2,16,Released,0.0
3,19,Released,24.0
4,20,Released,50.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,1.0,1.0,0.0,0.0,2.0
Executing,2.0,17.0,2.0,8.0,41.0
Released,22.0,2.0,0.0,24.0,50.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,1.0,1.0,0.0,0.0,2.0
1,Executing,2.0,17.0,2.0,8.0,41.0
2,Released,22.0,2.0,0.0,24.0,50.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,1.0,1.0,0.0,0.0,2.0
1,Executing,2.0,17.0,2.0,8.0,41.0
2,Released,22.0,2.0,0.0,24.0,50.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()